In [1]:
import os
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cv2

from OpenVisus import *
from io import BytesIO
from tqdm import tqdm

## Define some helper functions

In [2]:
def get_date_from_timestep(timestep: int) -> datetime.datetime:
    year = math.floor(timestep / 365)
    day = (timestep % 365)
    
    start_year = datetime.datetime(year=year, month=1, day=1)
    
    return start_year + datetime.timedelta(days=day)
    

## Define variables for the video

In [3]:
local_idx_file = '../NEX-GDDP-CMIP6_data/idx/day_ACCESS-CM2_historical_r1i1p1f1_gn.idx'
video_file_name = 'hurs_timestep_video.mp4'
frames_per_second = 6
field = 'hurs'
field_unit = '%'
field_full_name = 'Near-Surface Relative Humidity'
colormap = 'GnBu'

## Load the IDX file and create the video

In [4]:
idx_file = LoadDataset(local_idx_file)
time_steps = idx_file.getTimesteps()

# Flip dimensions for OpenCV
frame_size = (idx_file.shape[1], idx_file.shape[0])

video = cv2.VideoWriter(video_file_name, cv2.VideoWriter_fourcc(*'mp4v'), frames_per_second, frame_size)

SystemError: Error in swig director code what Visus throwing exception where(D:\a\OpenVisus\OpenVisus\Libs\Db\src\Dataset.cpp:311) what(empty content) where D:\a\OpenVisus\OpenVisus\build_windows\Release\VisusDbPyPYTHON_wrap.cxx : 29796 
 <no error?>

## Calculate the min / max of the values

In [5]:
max_value = 0
min_value = 0

first = True
for time_step in tqdm(time_steps):
    data = idx_file.read(field=field, time=time_step)
    
    if first:
        first = False
        min_value = np.nanmin(data)
        max_value = np.nanmax(data)
    else:
        min_value = min(min_value, np.nanmin(data))
        max_value = max(max_value, np.nanmax(data))
        
print(f'({min_value}, {max_value})')

NameError: name 'time_steps' is not defined

## Read all timesteps and plot them

In [ ]:
for time_step in tqdm(time_steps):
    data = idx_file.read(field=field, time=time_step)
    title = f'{field_full_name}: {get_date_from_timestep(time_step).date()}'
    extent = [0.125, 359.875, -59.875, 89.875]
    
    fig = plt.figure(figsize=(16, 6))
    axes = plt.axes(projection=ccrs.PlateCarree())
    plot = axes.imshow(data, aspect='auto', extent=extent, origin='lower', cmap=colormap, vmin=min_value, vmax=max_value)
    axes.coastlines()
    axes.gridlines(draw_labels=True)
    axes.set_xlabel('Longitude')
    axes.set_ylabel('Latitude')
    fig.suptitle(title)
    fig.colorbar(plot, location='right', pad=0.05, label=f'{field} ({field_unit})')
    
    buffer = BytesIO()
    plt.savefig(buffer, format='png')
    buffer.seek(0)
    
    # Close plot
    fig.clear()
    plt.close(fig)
    
    # Convert BytesIO to numpy array
    img_array = np.frombuffer(buffer.getvalue(), dtype=np.uint8)
    img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
    
    # Resize image to frame size and write to video
    img_resized = cv2.resize(img, frame_size)
    video.write(img_resized)
    
    break

video.release()